In [68]:
#更新dic_company字典表
# !/usr/bin/python
# -*- coding: utf-8 -*-
# Author: Selvaria

import requests
import time
from bs4 import BeautifulSoup
import csv
import pandas
import random
from selenium import webdriver
import pymysql
import concurrent.futures
from DBUtils.PooledDB import PooledDB
import datetime
import json
import re

now = datetime.datetime.now()
today = now.strftime('%Y-%m-%d')
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
DATE_FORMAT = "%m/%d/%Y %H:%M:%S %p"


_dbservice_pool = None

class MysqlPool(object): #设置数据库连接池和初始化    
    """
        MYSQL数据库对象，负责产生数据库连接 , 此类中的连接采用连接池实现
        获取连接对象：conn = Mysql.get_connection()
        释放连接对象;conn.close()或del conn
    """
    
    def __init__(self,mincached=5, maxcached=14,
                maxconnections=244, blocking=True, maxshared=0):
        """
            生成MySQL数据库连接池
        :param mincached: 最少的空闲连接数，如果空闲连接数小于这个数，pool会创建一个新的连接
        :param maxcached: 最大的空闲连接数，如果空闲连接数大于这个数，pool会关闭空闲连接
        :param maxconnections: 最大的连接数
        :param blocking: 当连接数达到最大的连接数时，在请求连接的时候，如果这个值是True，请求连接的程序会一直等待，
                          直到当前连接数小于最大连接数，如果这个值是False，会报错，
        :param maxshared: 当连接数达到这个数，新请求的连接会分享已经分配出去的连接
        """        
        db_config = {
            "host": '47.92.25.70',
            "port": 3306,
            "user": 'root',
            "passwd": 'Wfn031641',
            "db": 'cxd_data',
            "charset": 'utf8'
        }
        self.pool = PooledDB(pymysql, mincached=mincached, maxcached=maxcached, maxconnections=maxconnections,
                             blocking=blocking, maxshared=maxshared, **db_config)

    def get_connection(self):
        return self.pool.connection()

    def close(self):
        self.pool.close()

    def __del__(self):
        self.close()

# 获取mysql数据库服务器的连接
def get_dbservice_mysql_conn(): #实例化连接池类，方便调用
    """
    :return: Object  MySQL Connection
    """
    global _dbservice_pool
    if not _dbservice_pool or not isinstance(_dbservice_pool, MysqlPool):
        _dbservice_pool = MysqlPool()
    return _dbservice_pool.get_connection()


class QInfoAddition(object):
    
    headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36'
            }
    
    '''def __init__(self, name):
        self.name = name'''
    
    def get_login(self): #利用selenium手动登录,获取cookies
        url = 'https://www.qichacha.com/user_login'
        #CHROME_OPTIONS = webdriver.ChromeOptions()
        #CHROME_OPTIONS.add_argument(proxy)
        #browser = webdriver.Chrome(chrome_options=CHROME_OPTIONS)
        browser = webdriver.Chrome()
        browser.get(url) 
        time.sleep(30) #登录并录入验证码  #13342264161 #18698925437 #15524882680 ldp520615@ #13342270074 #13342270064 #13372864060 #13372864160
        cookies = browser.get_cookies()
        post={} #定义一个空的字典，存放cookies内容
        #获取到的cookies是列表形式，将cookies转成字典格式
        for cookie_item in cookies:
            post[cookie_item['name']] = cookie_item['value'] 
        browser.quit()
        cookie_str = json.dumps(post)
        return post,cookies #第一个是requests用，第二个是selenium用

    def get_company_code(self, cookies, name): #获取企业代号No（企查查id）
        company_name = name
        url_s = requests.get('https://www.qichacha.com/search?key=%s' %company_name, headers = self.headers, cookies = cookies).content
        soup_s = BeautifulSoup(url_s,'html.parser')
        try:
            num_u = soup_s.find('span', attrs = {'id': "countOld"})
            num = num_u.find('span', attrs = {'class': "text-danger"})
        except Exception as e:
            logging.warning('基本被封，请手动查询')
            print(str(e))
            #return None
            self.get_login()
            exit(1) # 有错误退出

        if num and num.get_text().strip() == '0':
            print('无此企业', name)
            return 0
        else:
            time.sleep(1)
            search_url = soup_s.find('a', attrs = {'class': "ma_h1"})['href']
            company_code = search_url[6:-5]
            #url_company = requests.get('https://www.qichacha.com%s' %search_url, headers = self.headers).content
            com_web_q = 'https://www.qichacha.com' + search_url #企查查的企业主页
            return company_code
        
    def get_com_status(self, code, cookies): #基础信息
        url = 'https://www.qichacha.com/firm_' + code +'.html'
        #print(url)
        url_company = requests.get(url, headers = self.headers, cookies = cookies).content
        soup_c = BeautifulSoup(url_company,'html.parser')
        try:
            company_title = soup_c.find('div', attrs = {'class': "row title jk-tip"}).find('h1').get_text() #企业名
            company_lead = soup_c.find('a', attrs = {'class': "bname"}).get_text() #法人
            company_tel_o = soup_c.find('span', attrs = {'class': "cvlu"}).get_text().strip()
            pattern = re.compile(r'\d+') 
            company_tel_list = pattern.findall(company_tel_o)
            if len(company_tel_list) > 0:
                company_tel = company_tel_list[0]
            else:
                company_tel = None            
            company_email = soup_c.find_all('span', attrs = {'class': "cvlu"})[2].get_text().strip()
            #print(company_lead, company_tel, company_email)
            company_table = soup_c.find_all('table', attrs = {'class': "ntable"})[1]
            table = company_table.find_all('td')
            data_sheet = []
            for i in table:
                data = i.get_text().strip()
                data_sheet.append(data)
            #print(len(data_sheet))
            #print(data_sheet)
            dict_d = {}
            for i in range(0,40,2):
                dict_d[data_sheet[i]] = data_sheet[i+1]
            #print(dict_d)
            found_cash = dict_d['注册资本']
            found_date = dict_d['成立日期']
            company_loc = dict_d['企业地址'].split('\n')[0]
            company_cover = dict_d['经营范围']
            company_tax_num = dict_d['纳税人识别号']
            company_prov = dict_d['所属地区']
            #print('公司名称：%s' %company_title)
            msg = [company_prov, company_title, company_lead, found_date, found_cash, company_loc, company_email, company_tel,\
                   company_cover, url, company_tax_num]
            print('%s web data is done.' %code)
            return msg
        except Exception as e:
            print(repr(e))
            print('网页数据提取出现问题')
    
    def get_addition_status(self, code, cookies): #补充信息，这个信息在招商库里
        url_t = 'https://www.qichacha.com/cbase_' + code
        url_l= 'https://www.qichacha.com/user_login'
        CHROME_OPTIONS = webdriver.ChromeOptions()
        CHROME_OPTIONS.add_argument('headless')
        browser = webdriver.Chrome(chrome_options=CHROME_OPTIONS)
        #browser = webdriver.Chrome() 
        browser.get(url_l) 
        for cookie_item in cookies:
            browser.add_cookie(cookie_item)
        browser.get(url_t)
        time.sleep(2)
        try:
            info_0 = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[2]/td[2]').text #经营状态
        except:
            info_0 = None
        if '在业' in info_0:
            info = 1
        elif '存续' in info_0:
            info = 2
        elif '注销' in info_0:
            info = 10
        elif '吊销' in info_0:
            info = 9
        elif '筹建' in info_0:
            info = 3
        elif '清算' in info_0:
            info = 4
        elif '迁入' in info_0:
            info = 5
        elif '迁出' in info_0:
            info = 6
        elif '停业' in info_0:
            info = 7
        elif '撤销' in info_0:
            info = 8
        elif '开业' in info_0:
            info = 1
        else:
            info = None
        #print(info)
        status_id = str(info)
        login_code = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[4]/td[2]').text #注册号
        party_code = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[4]/td[4]').text   #组织机构代码
        industry_part = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[5]/td[4]').text #所属行业
        login_position = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[6]/td[4]').text #注册地
        used_name = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[8]/td[2]').text #曾用名
        employer_amnt = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[9]/td[2]').text #人员规模
        insurance_num = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[8]/td[4]').text #参保人数
        runnint_time = browser.find_element_by_xpath('//*[@id="Cominfo"]/table[2]/tbody/tr[9]/td[4]').text #营业期限
        data = [status_id,login_code,party_code,industry_part,login_position,used_name,employer_amnt,insurance_num,runnint_time]
        data_handle = [None if '-' in x and len(x)==1 else x for x in data]
        browser.quit()
        return data_handle
        

def test(name):
    q = QInfoAddition()
    cookies_b, cookies_s = q.get_login()
    company_code = q.get_company_code(cookies_b, name)
    data = q.get_com_status(company_code, cookies_b)
    return data

test('浙江宏途物流有限公司')

a6a19385a48b121f5fda095dfb1c296f web data is done.


['浙江省',
 '浙江宏途物流有限公司',
 '张金荣',
 '2018-11-07',
 '500万元人民币',
 '浙江省丽水市莲都区水阁工业区祥龙街99号18幢104-2室(丽景民族工业园)',
 '暂无',
 None,
 '普通货物道路运输、货运代理;货运站(场)经营(货运配载、货运代理);仓储服务(不含危险化学品);汽车租赁服务;物流信息咨询服务;汽车及配件、电子产品、燃料油(不含成品油及危险化学品)销售;土石方工程专业承包(依法须经批准的项目,经相关部门批准后方可开展经营活动)',
 'https://www.qichacha.com/firm_a6a19385a48b121f5fda095dfb1c296f.html',
 '91331127MA2E0FBR5H']

In [5]:
connection = pymysql.connect(host='39.105.9.20', user='root', passwd='bigdata_oil',
                db='cxd_data', port=3306, charset='utf8')
sql_t = "select * from `financial_exchange` where `year` = '2019'"
df_t = pandas.read_sql(sql_t, connection)
#len(df_t)
df_t.head()

,financial_exchange_id,Xf_com_group,Xf_com_type,Xf_company_id,Xf_company_name,Gf_com_group,Gf_com_type,Gf_company_id,Gf_company_name,exchange_date,...,year,exchangetype,main_id,main_name,spec_id,spec_name,good_id,good_name,org_id,add_time
0,1435912,0.0,1.0,4355,山东齐成石油化工有限公司,1.0,2.0,2411,中油东港（大连）能源有限公司,2019-02-28,...,2019,1,None,None,None,None,None,None,None,NaT
1,2331407,1.0,2.0,2568,大连隆泰石油化工有限公司,0.0,1.0,16422,淄博市临淄鲁辉化工有限公司,2019-01-07,...,2019,1,None,None,None,None,None,None,None,2019-05-09 20:11:01
2,2331408,1.0,2.0,2568,大连隆泰石油化工有限公司,0.0,1.0,16422,淄博市临淄鲁辉化工有限公司,2019-01-07,...,2019,1,None,None,None,None,None,None,None,2019-05-09 20:11:01
3,2331409,1.0,2.0,2568,大连隆泰石油化工有限公司,0.0,1.0,16422,淄博市临淄鲁辉化工有限公司,2019-01-07,...,2019,1,None,None,None,None,None,None,None,2019-05-09 20:11:01
4,2331410,1.0,2.0,2568,大连隆泰石油化工有限公司,0.0,1.0,16422,淄博市临淄鲁辉化工有限公司,2019-01-07,...,2019,1,None,None,None,None,None,None,None,2019-05-09 20:11:01


In [24]:
q = QInfoAddition()
cookies_b, cookies_s = q.get_login()
company_code = q.get_company_code(cookies_b, '山东齐成石油化工有限公司')
data = q.get_com_status(company_code, cookies_b)
data

https://www.qichacha.com/firm_e810463a0819c7d05fc557a52bca0c2f.html
AttributeError("'NoneType' object has no attribute 'find'",)
网页数据提取出现问题


In [14]:
x_middle = df_t['Xf_company_name'].values
x_com = list(set(x_middle))
x_com

['贵阳方源酒店有限公司',
 '中油旭力（大连）石化有限公司',
 '中油运通（大连）石油化工有限公司',
 '雷州市茂华工贸公司加油站',
 '大连福庆石化有限公司',
 '大连晨和科技发展有限公司',
 '中国石化销售有限公司上海石油分公司',
 '中油领航石油化工（大连）有限公司',
 '中国石化销售有限公司山东济宁石油分公司',
 '张家港孚宝仓储有限公司',
 '大连鑫金达轮胎有限公司',
 '乌鲁木齐瑞龙加油站（有限公司）',
 '中国石化销售有限公司江苏南通石油分公司',
 '中油天翊（大连）石油化工有限公司',
 '温岭市九龙加油站(普通合伙)',
 '海城市宝顺运输有限公司',
 '江苏小米化工贸易有限公司',
 '盘锦瑞东石化贸易有限公司',
 '淄博驰程运输有限公司张店分公司',
 '上海桔程酒店管理有限公司',
 '大连屹峰石油化工有限公司',
 '常州市武进卜弋加油站有限公司',
 '安康市汉滨区驭力加油站',
 '嘉祥县西城加油站',
 '舟山远杰石化有限公司',
 '莱芜市泰德工贸有限公司',
 '赤峰平煤投资有限责任公司平庄宾馆',
 '中油天达（大连）石化有限公司',
 '中油万众石化（大连）有限公司',
 '中油海洋石油（大连）有限公司',
 '山东顺丰速运有限公司',
 '杭州皇冠大酒店',
 '沈阳拓扬商贸有限公司',
 '大连西中岛企业服务有限公司',
 '大连易思特化工有限公司',
 '嘉里建设管理（上海）有限公司沈阳分公司',
 '中储佳城能源(大连)有限公司',
 '郑州跃力货物运输有限公司',
 '中国石油天然气股份有限公司四川泸州销售分公司',
 '芜湖三山海螺港务有限公司',
 '邢台佳通危货运输有限公司',
 '辽阳市嘉驿危险货物运输有限责任公司',
 '大连汽贸集团顺翔汽车销售服务有限公司',
 '广州市中瀚船舶燃料供应有限公司',
 '南京钢铁四通运输有限责任公司',
 '辽宁滨海通力石油化工有限公司',
 '中油天南石化（大连）有限公司',
 '辽宁如家奥福特精选酒店管理有限公司',
 '郑州利来运输有限公司',
 '中油三维丝石化（大连）有限公司',
 '中国石化销售有限公司安徽黄山市黄山区石油分公司',
 '中油鑫汇（大连）石化有限公司',
 '武义县王路头加油点',
 '大连惠海石油化工有限公司'

In [41]:
def check_company(com):
    connection = pymysql.connect(host = '47.92.25.70',user = 'root',password = 'Wfn031641',db = 'cxd_data',charset = 'utf8')
    try:
        with connection.cursor() as cursor: 
            sql = "select `id` from `dic_company` where `name` = '{}'".format(com)
            cursor.execute(sql)
            cover = cursor.fetchone()[0]
        connection.close()
    except:
        cover = False
    return cover

In [42]:
xf_company_new = []
for com in x_com:
    result = check_company(com)
    if result == False:
        xf_company_new.append(com)
    elif len(com) < 2:
        continue
    else:
        continue
xf_company_new

['贵阳方源酒店有限公司',
 '中油旭力（大连）石化有限公司',
 '雷州市茂华工贸公司加油站',
 '大连晨和科技发展有限公司',
 '中国石化销售有限公司山东济宁石油分公司',
 '大连鑫金达轮胎有限公司',
 '乌鲁木齐瑞龙加油站（有限公司）',
 '温岭市九龙加油站(普通合伙)',
 '海城市宝顺运输有限公司',
 '淄博驰程运输有限公司张店分公司',
 '上海桔程酒店管理有限公司',
 '大连屹峰石油化工有限公司',
 '常州市武进卜弋加油站有限公司',
 '安康市汉滨区驭力加油站',
 '嘉祥县西城加油站',
 '舟山远杰石化有限公司',
 '莱芜市泰德工贸有限公司',
 '中油天达（大连）石化有限公司',
 '杭州皇冠大酒店',
 '沈阳拓扬商贸有限公司',
 '大连易思特化工有限公司',
 '中储佳城能源(大连)有限公司',
 '郑州跃力货物运输有限公司',
 '中国石油天然气股份有限公司四川泸州销售分公司',
 '大连汽贸集团顺翔汽车销售服务有限公司',
 '广州市中瀚船舶燃料供应有限公司',
 '辽宁滨海通力石油化工有限公司',
 '辽宁如家奥福特精选酒店管理有限公司',
 '郑州利来运输有限公司',
 '中油三维丝石化（大连）有限公司',
 '中国石化销售有限公司安徽黄山市黄山区石油分公司',
 '武义县王路头加油点',
 '江苏省供销社苏北油库',
 '辽宁顺安贸易有限公司',
 '遵化市千里行加油加气站',
 '江苏九季酒店管理有限公司',
 '青岛思芮雅科技技术有限公司',
 '中油中顺石油化工（大连）有限公司',
 '辽宁省高速公路实业发展有限责任公司塔山服务区',
 '南漳县浩天商贸有限公司加油点',
 '淄博蓝海国际大饭店有限公司',
 '中国石化销售有限公司山东枣庄石油分公司',
 '平江县鸿源加油站',
 '中油富杰石油化工（大连）有限公司',
 '新乡市龙浩实业有限公司陈堡加油站',
 '中国石化销售有限公司贵州黔南瓮安石油分公司',
 '大连旅顺供销大厦有限公司',
 '枣庄市山亭区圣源加油站',
 '山东温暖之家商务服务有限公司济宁市兖州区永华舜耕国际酒店',
 '中国石化销售股份有限公司黑龙江哈尔滨石油分公司',
 '常德市鼎城瑞兴燃气有限责任公司',
 '大连万格方元能源有限公司',
 '中化道达尔油品有限公司苏州工

In [43]:
print(len(x_com))
print(len(xf_company_new))

2159
1024


In [47]:
g_middle = df_t['Gf_company_name'].values
g_com = list(set(g_middle))
gf_company_new = []
for com in g_com:
    result = check_company(com)
    if result == False:
        gf_company_new.append(com)
    elif len(com) < 2:
        continue
    else:
        continue
print(gf_company_new)
print(len(g_com))
print(len(gf_company_new))

['', '盐城大丰迅尔达物流有限公司', '北京八维研修学院', '江西盛和远物流有限公司', '浙江宏途物流有限公司', '东营德鹏商贸有限公司', '芜湖县清发货物运输有限公司', '广州腾通物流有限公司', '蓬安鸿宇运业有限责任公司', '鑫峦贸易(上海)有限公司', '青岛圣凡物流有限公司', '河北万晔科技发展有限公司', '大连家鸿物资有限公司', '合肥路甲物流有限公司', '宝清县福城众兴煤炭经销有限公司', '长春市越程物流有限公司', '龙游鑫泉机械设备租赁有限公司', '沈阳鼎宏科技有限公司', '汾阳市胜达物流有限公司', '辽宁滨海通力石油化工有限公司', '大连远洲大酒店有限公司', '江苏驰畅建设有限公司', '忻州市晋龙汽车运输服务有限公司', '天津市滨海新达化工厂（普通合伙）', '中铁六局集团北京铁路建设有限公司', '南通易大宗供应链管理有限公司', '东莞市中堂金星静电喷塑厂', '山东福斯润滑油有限公司', '中交第一航务工程局有限公司广西分公司', '龙江县悦丰运输有限公司', '天津宇昊建设工程集团有限公司', '马鞍山旻尧物流有限公司', '武汉炎隆义石化有限公司', '广州港之龙物流有限公司', '江西千彩水晶有限公司', '浙江舟山易能石油化工有限公司', '合肥国俊供应链管理有限公司', '洛阳惠中兽药有限公司', '大商股份有限公司瓦房店炮台大商新城镇购物中心', '河南皓飞化工产品有限公司', '大连旅顺供销大厦有限公司', '中山市中申物流有限公司', '酒泉市浩海煤化有限公司', '石家庄中宇通化工有限公司', '辽阳县中安矿业有限责任公司', '洪洞县安信恒顺煤业有限公司', '铜陵东林航运有限公司', '青岛港盛物业管理有限公司', '瓦房店科锐达轴承机械制造有限公司', '长春市日鑫物流有限公司', '广西金祥化工科技有限公司', '大连君创能源有限公司', '上海熊郑实业有限公司', '合肥沃隆物流有限公司', '南京鼎冠锦纶新材料有限公司', '盘锦鸿聚石油化工有限公司', '大庆市蓝帆科技有限责任公司', '德惠市鹏运物流有限公司', '湖北润德西科技有限公司', '揭阳市恩乐斯塑胶有限公司', '唐山奥铁贸易有限公司', '宁阳青山物流有限公司兴通分公司', '青岛华中建安建筑工程有限

In [48]:
up_com_list = gf_company_new + xf_company_new
len(up_com_list)

5333

In [53]:
def up_sql_company(data): #上传数据库dic_company
    connection = get_dbservice_mysql_conn()
    if '长兴岛' in data[5]:
        try :
            with connection.cursor() as cursor:
                sql_i = "INSERT INTO `dic_company` (`name`,`com_group`,`company_province`,`company_provinceId`,`company_city`,`company_cityId`)\
                VALUES(%s,%s,%s,%s,%s,%s)"
                cursor.execute(sql_i,[data[1], 1, data[0], 8, '大连市', 51])
            connection.commit() 
            #print('成功')
        except pymysql.err.IntegrityError as e:
            print('出现重复值，进行覆盖',e)
        except Exception as e:
            print(e)
    else:
        try :
            with connection.cursor() as cursor:
                sql_i = "INSERT INTO `dic_company` (`name`,`com_group`,`company_province`) VALUES(%s,%s,%s)"
                cursor.execute(sql_i,[data[1], 0, data[0]])
            connection.commit() 
            #print('成功')
        except Exception as e:
            print(e)
        finally:
            connection.close()
    print('"dic_company" Single is Done')
    
def get_id_data(data):
    connection = get_dbservice_mysql_conn()
    with connection.cursor() as cursor: 
        sql = "select `id` from `dic_company` where `name` = '{}'".format(data[1])
        cursor.execute(sql)
        c_id = cursor.fetchone()[0]
    connection.close()
    data.pop(0)
    data.insert(-1, c_id)
    return data

def up_sql_detail(data): #上传数据库dic_company_detail
    connection = get_dbservice_mysql_conn()
    try :
        with connection.cursor() as cursor:
            sql_i = "insert into `dic_company_detail`(`name`,`person`,`date`,`capital`,`address`,`email`,`phone`,`operate`,`website`,\
            `company_id`, `tax_num`)\
            values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql_i,data)
        connection.commit() 
        #print('成功')
    except Exception as e:
        print(e)
    finally:
        connection.close()
    print('"dic_company_detail" Single is Done')

In [ ]:
test_com = up_com_list[1]
#print(test_com)

q = QInfoAddition()
cookies_b, cookies_s = q.get_login()

error_list = []

for com in up_com_list[2725:5333]:
    com_index = up_com_list.index(com)
    company_code = q.get_company_code(cookies_b, com)
    print('开始进行"%s"公司，是第%s个' %(com,com_index))
    try:
        data = q.get_com_status(company_code, cookies_b)
        up_sql_company(data)
    except Exception as e:
        print('%s出现问题' %com)
        print(str(e))
        error_list.append(com)
        continue
    data_detail = get_id_data(data)
    up_sql_detail(data)
    time.sleep(14)

开始进行"含山县润发物流有限公司"公司，是第970个
6e028d3d53fd97ad6eb5264ebb8ac114 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"洛南县三江天然气有限公司"公司，是第971个
0bd334592c07cfb5e1384d499d8bd2db web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"大连保税区莹达物流有限公司"公司，是第972个
fc272d28576c28b376967a722c07f694 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"北京现代金宇岩土工程有限公司"公司，是第973个
8536febdf2be1e7c33d7ae7eb2ea9eb0 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"仁县荣泰物流有限公司含山运漕分公司"公司，是第974个
9546cf59ac42c5ac7bc0f667ca2efdc3 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"邢台市华熙橡塑制品有限公司"公司，是第975个
442e8f13c9f19ffcd074f826e8ac8c01 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"锦州市绿源危险废物处置有限公司"公司，是第976个
240bb118f9cdff5b72f10d2f115a7704 web data is done.
"dic_company" Single is Done
"dic_company_deta

D:\Selvaria\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'operate' at row 1")
  result = self._query(query)


"dic_company_detail" Single is Done
开始进行"东莞市太一生物科技有限公司"公司，是第980个
ce22152b006ee4445266af65fc4b0027 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"潍坊龙顺物流有限公司"公司，是第981个
8f5db7d6f14e8ab4ea5b2bb83950840e web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"定兴县京华货运有限公司"公司，是第982个
a5d38c09d05a9094ab1373addaeea920 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"衢州君诚物流有限公司"公司，是第983个
c3e3dab9ebc715a7d631ed4cfb04ac3d web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"吉林省雅洋能源投资有限责任公司开运街加油站"公司，是第984个
cebb4482d127aaab75c55355640fc9ec web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"长春市铁成奥祥物流有限公司"公司，是第985个
406b2a4549d5ba39cc3ce74dac22bd99 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"日照北联石油化工有限公司"公司，是第986个
95181c1d2ede06d79fd5d47d82e5352a web data is done.
"dic_company" S

"dic_company_detail" Single is Done
开始进行"湖北尚美鲜商贸有限公司"公司，是第1037个
2132a6e6eb5364dc6039c2cea34e5ee9 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"北京源泰隆物流有限公司"公司，是第1038个
edcb979e6e5ce9019b321e8ae1231268 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"贵州浩品建筑工程有限公司务川分公司"公司，是第1039个
509f220dffcae6de33fbf58794f3d648 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"武穴市大明化工建材有限公司"公司，是第1040个
e105160ab84048f55c9d774e72e9679f web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"宁波金屿化工有限公司"公司，是第1041个
3f8b21317668725069cee6261f18bef1 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"云南华银园艺环境科技有限公司"公司，是第1042个
97569b6b7de53e882d18288e93d1be9d web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"庞大乐业租赁有限公司"公司，是第1043个
982b043d668972e8c4c3f1670b87aac1 web data is done.
"dic_company

"dic_company_detail" Single is Done
开始进行"海阳市连福制衣有限公司"公司，是第1093个
afe588715a8ac246aa22cf08f7ee4236 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"广州市琦胜化工有限公司"公司，是第1094个
36c14c25be87689219092a39379d78f0 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"深圳恒泰达物流有限公司"公司，是第1095个
c0c450ff2fdd21bd8cb1e27fb9f9551c web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"郑州优尼凯贸易有限公司			"公司，是第1096个
51e358f6c67fb2d947f4f2d3167d7b88 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"东营市东泽化工科技有限公司"公司，是第1097个
80233ae66c89d9669c7aef0df240f3dc web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"联合汽车电子有限公司重庆分公司"公司，是第1098个
8e55ca8e7dcc3252cb4972f9cc60f8d8 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"中油陆港能源（大连）有限公司"公司，是第1099个
11a28a19673650b8deac2ec58692fec7 web data is done.
"dic_comp

...

2d5a9251608b4ffc5cef19c143031f69 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"上海飘焕实业有限公司"公司，是第2504个
53c90c394801d1a890da0fce285dbf1d web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"四川嘉顺恒石化有限公司"公司，是第2505个
e561891657ad7450a6ba4f6c94104e7a web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"大连华南广益供应链管理有限公司"公司，是第2506个
0d9ba6bcb49b881c1f772462860d2a87 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"四川鼎信中汇石油化工有限责任公司	"公司，是第2507个
5877795019a8efc9900e3b32d154c612 web data is done.
(1062, "Duplicate entry '四川鼎信中汇石油化工有限责任公司' for key 'name'")
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"浙江海力股份有限公司"公司，是第2508个
KeyError('注册资本',)
网页数据提取出现问题
浙江海力股份有限公司出现问题
'NoneType' object is not subscriptable
开始进行"丹东东港经济开发区物业有限公司六合加油站"公司，是第2509个
765802a938418692836c87b7fa2b252a web data is done.
(1062, "Duplicate entry '丹东市东港经济开发区物业有限公司

89733261f20d7ba3cc453f4af266a6ab web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"成都惠捷物流有限公司"公司，是第2560个
8ec521fbf1bcf711e02464c29c60fb1c web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"安徽全直达供应链管理有限公司"公司，是第2561个
cbdc5137372f31836d64414e9eba66a1 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"吉林省蚁巢物流运输有限公司"公司，是第2562个
95043d0b746b7ff71a9b65aa9e59d697 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"山东纳恩生物科技有限公司"公司，是第2563个
35ecaf0f16e06db9dfeb6762f8b399cf web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"北京快行线冷链物流有限公司"公司，是第2564个
aae6794ee3ae2fc7e8836e0d60eb3c97 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"辽阳天顺商贸有限公司"公司，是第2565个
1027fd6b34e17faffb58d3b3d10a6938 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"十堰旭鑫新能源有限

358d5e1c5cb68221585d5d66eb0a7aee web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"常州市千里马运输有限责任公司"公司，是第2616个
90ca21ef3ee30ca8980da6b82121d10a web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"唐山东联运输服务有限公司"公司，是第2617个
9189e7e15e0a5b3733109440d70bafe2 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"宽城满族自治县顺达加油站"公司，是第2618个
2529278761266b6c026c60627782935b web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"山东坤少石油化工有限公司"公司，是第2619个
4f8c8e7648a04fbfd8413f7213ec4008 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"山西中兴铸业集团有限公司"公司，是第2620个
7946713c5f578cacf5db1ee9da7b222b web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"云南金沙矿业股份有限公司"公司，是第2621个
9761c0f112e059b1ee654ac30f471399 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"营口华源石化有

"dic_company_detail" Single is Done
开始进行"河北坤天新能源科技有限公司"公司，是第2672个
4695fe6f244ace8e11a8c5327b814ba1 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"浙江自贸区朝宇石化有限公司"公司，是第2673个
195b73fd81a0c1dbeb9e5655019d741e web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"合肥诗好物流有限公司"公司，是第2674个
3bd882cfd31da6601bc9deb27d8c2fe4 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"庐江县白湖大桥加油站"公司，是第2675个
1dd421ec6554dea205c3e5afa5bdb5a6 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"大连誉铭建材有限公司"公司，是第2676个
70d44ec7397b15a8025e3ca71eb25bbf web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"揭阳市帆琳玩具有限公司"公司，是第2677个
3e8b6d2f0a9b42f4f2cceda112e78d8b web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"丹东顺捷物流有限公司"公司，是第2678个
ff8be97acd53b90535af3901f89d7627 web data is done.
"dic_company" Single 

"dic_company_detail" Single is Done
开始进行"深圳市骏佳物流有限公司"公司，是第2728个
3587d82adb2223fef15f3bacfb4657b4 web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done
开始进行"辽宁省石油总公司凌海市公司南山加油站"公司，是第2729个
6be4e7d810494ae9023f6c76316dcd3b web data is done.
"dic_company" Single is Done
"dic_company_detail" Single is Done


In [67]:
import re

text_tel = '更多号码'

pattern = re.compile(r'\d+') #寻找任意长度整数
price = pattern.findall(text_tel)
price

[]